## Calculating trial-wise parametric modulations

This script calculates values of model variables like value estimation or prediction error for each trial. Script features:
- loads point estimations of model parameters (e.g. learning rates)
- calculates trial-wise estimates of different model tracking variables (e.g. prediction error)
- saves these parametric modulations to file

(*Note*) Usually subjects are trying to minimize overal prediction error. Here it is not precisely the case, because sometimes subject pick probabilistically worse option to maximize their winnings.

---
**Last update**: 07.02.2020 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sys
import os

path_root = os.environ.get('DECIDENET_PATH')
path_code = os.path.join(path_root, 'code')
if path_code not in sys.path:
    sys.path.append(path_code)

from dn_utils.behavioral_models import (load_behavioral_data, 
    estimate_wbci_pd, estimate_modulation)
from dn_utils.plotting import plot_trial_modulation

In [ ]:
# Directory to save modulations
path_out = os.path.join(path_root, 'data/main_fmri_study/derivatives/nistats/modulations')
os.makedirs(path_out, exist_ok=True)

# Load behavioral data
path_beh = os.path.join(path_root, 'data/main_fmri_study/sourcedata/behavioral')
beh, meta = load_behavioral_data(path_beh)
n_subjects, n_conditions, n_trials, _ = beh.shape

# Load point estimates for learning rates
path_param = os.path.join(path_code, 'behavioral_analysis/matjags-dn/data/' + \
                                     'subjects_point_estimates/alpha_pdci_mle.npy')
alpha_pdci_mle = np.load(path_param)

### Calculate and save parametric modulations.
Here parametric modulations reflecting three relevant model variables are calculated:
- `modulations_wcor`: parametric modulation during choice phase reflecting estimated probability of choosing correct (winning / not loosing) box
- `modulations_exvl`: parametric modulation during choice phase reflecting Pascalian expected value
- `modulations_perr`: parametric modulation during outcome phase reflecting prediction error on box probability 
    - equals `1 - wcor` for correct anwers
    - equals `0 - wcor` for incorrect answers

In [ ]:
modulations_wcor = np.zeros((n_subjects, n_conditions, n_trials))
modulations_exvl = np.zeros((n_subjects, n_conditions, n_trials))
modulations_perr = np.zeros((n_subjects, n_conditions, n_trials))

for sub in range(n_subjects):
    for con in range(n_conditions):
        
        wbci = estimate_wbci_pd(
            beh, meta, sub, con,
            alpha_plus=alpha_pdci_mle[sub, 0], 
            alpha_minus=alpha_pdci_mle[sub, 1]
        )
        
        wcor, exvl, perr = estimate_modulation(beh, meta, sub, con, wbci)
        
        modulations_wcor[sub, con, :] = wcor
        modulations_exvl[sub, con, :] = exvl
        modulations_perr[sub, con, :] = perr

np.save(os.path.join(path_out,'modulations_wcor'), modulations_wcor)
np.save(os.path.join(path_out,'modulations_exvl'), modulations_exvl)
np.save(os.path.join(path_out,'modulations_perr'), modulations_perr)

In [ ]:
# Plot example regressors
sub, con = 1, 0

plot_trial_modulation(beh, meta, sub, con, modulations_perr[sub, con, :])
plot_trial_modulation(beh, meta, sub, con, modulations_wcor[sub, con, :])
plot_trial_modulation(beh, meta, sub, con, modulations_exvl[sub, con, :])